In [1]:
import os.path as path
import sys

import tensorflow as tf

from src.data.requests_downloader import UrlDownloader
from src.models.model_builder import Model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

print(f"TensorFlow Version: {tf.__version__}")
print()
print(f"Python {sys.version}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


PROJECT_PATH = UrlDownloader().get_project_root()
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

def m2tex(model: tf.keras.Model, modelName: str, line_length: int):
    """
    Source: https://github.com/intentodemusico/py2Tex
    """
    stringlist = []
    model.summary(line_length=line_length, print_fn=lambda x: stringlist.append(x))
    del stringlist[1:-4:2]
    del stringlist[-1]
    first_vertical_border = int(31 / 70 * line_length)
    second_vertical_border = int(59 / 70 * line_length)
    for ix in range(1, len(stringlist) - 3):
        tmp = stringlist[ix]
        stringlist[ix] = tmp[0:first_vertical_border]+"& "+tmp[first_vertical_border:second_vertical_border]+"& "+tmp[second_vertical_border:]+"\\\\ \hline"
    stringlist[0] = "Model: {} \\\\ \hline".format(modelName)
    stringlist[1] += " \hline"
    stringlist[-4] += " \hline"
    stringlist[-3] += " \\\\"
    stringlist[-2] += " \\\\"
    stringlist[-1] += " \\\\ \hline"
    prefix = ["\\begin{table}[]", "\\begin{tabular}{lll}"]
    suffix = ["\end{tabular}", "\caption{{Model summary for {}.}}".format(modelName), "\label{tab:model-summary}" , "\end{table}"]
    stringlist = prefix + stringlist + suffix
    out_str = " \n".join(stringlist)
    out_str = out_str.replace("_", "\_")
    out_str = out_str.replace("#", "\#")
    return out_str

TensorFlow Version: 2.5.0

Python 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
GPU is NOT AVAILABLE


In [7]:
version = "original"
model = Model(
    "None",
    batch_size=4,
    input_image_height=512,
    input_image_width=512,
    number_of_classes=5,
    model_architecture=version
).get_deeplab_model()

export_path = path.join(PROJECT_PATH, f"./data/architectures/{version}.png")
plot_model(model, to_file=export_path, show_shapes=True,show_dtype=False, expand_nested=True, dpi=500)
plt.close()

In [15]:
# Specify the indices of the layers you want to include in the summary
selected_layer_indices = range(2, 20)

# Create a new model with only the selected layers
selected_layers_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[model.layers[i].output for i in selected_layer_indices]
)

print(m2tex(selected_layers_model, f"Deeplabv3plus {version}", 105))

\begin{table}[] 
\begin{tabular}{lll} 
Model: Deeplabv3plus original \\ \hline 
Layer (type)                                  &  Output Shape                             &  Param \#         \\ \hline \hline 
input\_2 (InputLayer)                          &  [(None, 512, 512, 3)]                    &  0               \\ \hline 
entry\_flow\_conv1\_1 (Conv2D)                   &  (None, 256, 256, 32)                     &  864             \\ \hline 
entry\_flow\_conv1\_1\_BN (BatchNormalization)    &  (None, 256, 256, 32)                     &  128             \\ \hline 
activation\_87 (Activation)                    &  (None, 256, 256, 32)                     &  0               \\ \hline 
entry\_flow\_conv1\_2 (Conv2D)                   &  (None, 256, 256, 64)                     &  18432           \\ \hline 
entry\_flow\_conv1\_2\_BN (BatchNormalization)    &  (None, 256, 256, 64)                     &  256             \\ \hline 
activation\_88 (Activation)                    &  (None,

In [6]:
text_file = path.join(PROJECT_PATH, f"./data/architectures/{version}.txt")
sys.stdout = open(text_file, 'w')
model.summary()
sys.stdout.close()
sys.stdout = sys.__stdout__